In [ ]:
%matplotlib inline
import numpy as np
import pylab as pl

from psi.controller.calibration.util import load_calibration
from psi.controller.calibration import tone
from psi.core.enaml.api import load_manifest_from_file

frequencies = [250, 500, 1000, 2000, 4000, 8000, 16000, 32000]
#frequencies = [8000]

io_file = 'c:/psi/io/pika.enaml'
cal_file = 'c:/psi/io/pika/default.json'

io_manifest = load_manifest_from_file(io_file, 'IOManifest')
io = io_manifest()
audio_engine = io.find('NI_audio')

channels = audio_engine.get_channels(active=False)
load_calibration(cal_file, channels)

mic_channel = audio_engine.get_channel('microphone_channel')
mic_channel.gain = 40
print(mic_channel.calibration)

speaker_channel = audio_engine.get_channel('speaker_1')
print(speaker_channel.calibration)

In [ ]:
fixed_gain_result = tone.tone_sens(
    frequencies=frequencies,
    engine=audio_engine,
    ao_channel_name='speaker_1',
    ai_channel_names=['microphone_channel'],
    gains=-30,
    debug=True,
    duration=0.1,
    iti=0,
    trim=0.01,
    repetitions=2,
)

In [ ]:
figure, axes = pl.subplots(4, 2)

for ax, freq in zip(axes.ravel(), frequencies):
    w = fixed_gain_result.loc['microphone_channel', freq]['waveform'].T
    ax.plot(w)
    ax.set_title(freq)

In [ ]:
from psi.controller.calibration.api import PointCalibration

r = fixed_gain_result.loc['microphone_channel']
calibration = PointCalibration(r.index, r['sens'])
gains = calibration.get_gain(frequencies, 80)

In [ ]:
variable_gain_result = tone.tone_spl(
    frequencies=frequencies,
    engine=audio_engine,
    ao_channel_name='speaker_1',
    ai_channel_names=['microphone_channel'],
    gains=gains,
    debug=True,
    duration=0.1,
    iti=0,
    trim=0.01,
    repetitions=2,
)

variable_gain_result['spl']